# Чтение и запись данных. Часть 2

## 1. Работа с базой данных из pandas


### 1.1 

Подготовка данных:

In [3]:
import psycopg2
import pandas as pd
import sqlite3

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)

cursor = conn.cursor()

# получаем имена таблиц из базы
sql_str = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
cursor.execute(sql_str)
tables_data = [a for a in cursor.fetchall()]
conn.commit()

print("Какие таблицы содержатся в Postgres: %s" % tables_data)

df = pd.read_sql_query("select * from public.ratings limit 5;", conn)
df.head()

Какие таблицы содержатся в Postgres: [('keywords',), ('links',), ('ratings',), ('exploratory',), ('course_purchases',)]


,userid,movieid,rating,timestamp
0,6854,832,5.0,857934552
1,6854,1356,5.0,857934470
2,6855,260,5.0,1048000350
3,6855,356,4.0,1047999391
4,6855,459,3.0,1047999486


**Задание простого уровня** <br>Посчитайте количество записей в каждой из таблиц **keywords, links, ratings** и выведите в формате имя *таблицы:количество строк*

*Результат работы:*
<pre>
table keywords:92838
table links:91686
table ratings:1555552
</pre>

In [4]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

table_names = [i[0] for i in tables_data]
for table in table_names:
        # -- ВАШ КОД ТУТ --
        sql_str = "SELECT count(*) FROM public.{} LIMIT 1;".format(table)
        cursor.execute(sql_str)
        row_count = [a for a in cursor.fetchall()][0][0]
        print("table {}:{}".format(table, row_count))

cursor.close()

table keywords:92838
table links:91686
table ratings:1555552
table exploratory:100
table course_purchases:154


### 1.2

**Задание среднего уровня** <br>Напечатайте колонки таблицы *ratings*. Подсказка: поисследуйте объект **cursor** с помощью встроенной в python команды *dir()*

In [5]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

sql_str = "SELECT * FROM ratings LIMIT 10;"
cursor.execute(sql_str)
conn.commit()

print(dir(cursor))
# print(cursor.description)

# -- ВАШ КОД ТУТ --
# table_rating_fields = []
table_rating_fields = [i.name for i in cursor.description]
print("Поля таблицы ratings: %s" % table_rating_fields)

cursor.close()

['__class__', '__delattr__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'arraysize', 'binary_types', 'callproc', 'cast', 'close', 'closed', 'connection', 'copy_expert', 'copy_from', 'copy_to', 'description', 'execute', 'executemany', 'fetchall', 'fetchmany', 'fetchone', 'itersize', 'lastrowid', 'mogrify', 'name', 'nextset', 'pgresult_ptr', 'query', 'row_factory', 'rowcount', 'rownumber', 'scroll', 'scrollable', 'setinputsizes', 'setoutputsize', 'statusmessage', 'string_types', 'typecaster', 'tzinfo_factory', 'withhold']
Поля таблицы ratings: ['userid', 'movieid', 'rating', 'timestamp']


### 1.3

**Задание высокого уровня**
* Посмотрите на столбцы таблицы **links** и столбцы таблицы **ratings** по какому полю можно заджойнить эти две таблицы
* Посчитайте количество фильмов из links, у который нет оценок в ratings.

In [6]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()


# -- ВАШ КОД ТУТ --
sql_str = """
    SELECT COUNT(DISTINCT links.movieId)
    FROM links;
"""

cursor.execute(sql_str)
row_count = [a for a in cursor.fetchall()][0][0]
conn.commit()

print("Количество фильмов без рейтингов: %s" % row_count)

sql_str = "select * from links limit 5;"
cursor.execute(sql_str)
print("Таблицв links:\n %s" % [i.name for i in cursor.description])
for row in cursor: print(row)

sql_str = "select * from ratings limit 5;"
cursor.execute(sql_str)
print("Таблица ratings:\n %s" % [i.name for i in cursor.description])
for row in cursor: print(row)

sql_str = """
SELECT count(*)
FROM   (SELECT count(ratings.movieid) as links_count
        FROM links
        LEFT JOIN ratings ON links.movieid = ratings.movieid
        GROUP BY links.movieid
        ORDER BY links_count
        ) AS links_count;
    """
cursor.execute(sql_str)
print("Таблицв query:\n %s" % [i.name for i in cursor.description])
for row in cursor: print(row)    
print(f'Всего {row[0]} фильмов не имеют рейтинга')
    
cursor.close()

Количество фильмов без рейтингов: 45843
Таблицв links:
 ['movieid', 'imdbid', 'tmdbid']
(1, '0114709', '862')
(2, '0113497', '8844')
(3, '0113228', '15602')
(4, '0114885', '31357')
(5, '0113041', '11862')
Таблица ratings:
 ['userid', 'movieid', 'rating', 'timestamp']
(6854, 832, 5.0, 857934552)
(6854, 1356, 5.0, 857934470)
(6855, 260, 5.0, 1048000350)
(6855, 356, 4.0, 1047999391)
(6855, 459, 3.0, 1047999486)
Таблицв query:
 ['count']
(45843,)
Всего 45843 фильмов не имеют рейтинга


## 2. Работа с БД: MongoDB

### 2.1

Подготовка данных

In [8]:
from pymongo import MongoClient

mongo_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 27017,
    "user": "students",
    "password": "X63673t47Gl03Sq",
    "authSource": "movies"
}

mongo = MongoClient('mongodb://%s:%s@%s:%s/?authSource=%s' % (
    mongo_connection['user'], mongo_connection['password'],
    mongo_connection['host'], mongo_connection['port'], mongo_connection['authSource'])
)
db = mongo["movies"]

print("Коллекции, доступные в MongoDB: %s" % db.list_collection_names())

collection = db['tags']
print("Число документов в коллекции %s" % collection.estimated_document_count())

Коллекции, доступные в MongoDB: ['tags', 'users']
Число документов в коллекции 158680


**Задание простого уровня** <br>Посчитайте количество контента, у которого встречается тэг "toy"

* сделайте выборку такого контента (а точнее - поля `id`) в питоновский `list`
* подсчитайте количество элементов в `list`

In [9]:
selector = {'name': 'toy'}
exclude_id = {'_id': False}

mongo_cursor = collection.find(projection=exclude_id, filter={'name': 'toy'})

# -- ВАШ КОД ТУТ --
# print(type(mongo_cursor), [item for item in mongo_cursor])
# cursor_items = set([])
cursor_items = []
for item in mongo_cursor:
    cursor_items.append(item['id'])
#     print(item)
print(cursor_items)
print("Количество контента с тэгом 'toy': %s\n" % len(cursor_items))

[4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290, 4290]
Количество контента с тэгом 'toy': 19



### 2.2

**Задание среднего уровня** <br> Выведите **только названия** 10 самых непопулярных тэгов <br>

Подсказка: посмотрите в уроке, как MongoDB позволяет также выполнять сложные агрегирующие запросы средствами СУБД

*Результат работы*
<pre>
Счётчик тэгов ['ancient tablet', 'remains', 'robespierre', 'social scandal', 'brain implant', 'adam west', 'arm cast', 'acab', 'gas explosion', 'female psychologist']
</pre>

In [ ]:
# -- ВАШ КОД ТУТ --
collection = db['tags']

pipline = [
    {"$group":
        {"_id": "$name",
         "tag_count":
            {"$sum": 1}
         }
     },
    {"$sort":
        {"tag_count": 1}
     },
    {"$limit": 10}
]

print([i['_id'] for i in collection.aggregate(pipline)])


### 2.3

**Задание высокого уровня**

1. Подключитесь к Postgres. Найдите top-10 самого популярного контента
1. Сохраните в файл `content_popularity.json` в виде
<pre>
{'movieId': 931, 'popularity': 999}
{'movieId': 429, 'popularity': 111}
</pre>

Этот формат называется single-line json и его можно загрузить в MongoDB с помощью команды
<pre>
mongoimport --host $APP_MONGO_HOST --port $APP_MONGO_PORT --db movies --collection tags --file content_popularity.json
</pre>

**ВНИМАНИЕ!** Это просто пример как можно использовать single-line JSON, заливать его никуда не нужно - в задаче требуется только подготовить файл в правильном виде

*Подсказка* воспользуйтесь функцией `json.dumps`

*Результат работы* - файл `content_popularity.json`:

<pre>
{"movieId": 318, "popularity": 5626}
{"movieId": 356, "popularity": 5464}
{"movieId": 296, "popularity": 5242}
{"movieId": 593, "popularity": 5048}
{"movieId": 2571, "popularity": 4718}
{"movieId": 260, "popularity": 4556}
{"movieId": 480, "popularity": 4540}
{"movieId": 527, "popularity": 4120}
{"movieId": 1, "popularity": 4072}
{"movieId": 110, "popularity": 3928}
</pre>

In [ ]:
mkdir compute

In [ ]:
# -- ВАШ КОД ТУТ --
import json
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

sql_str = """
    SELECT movieid AS movieId, COUNT(movieid) AS popularity
    FROM ratings
    GROUP BY movieId
    ORDER BY popularity DESC
    LIMIT 10;
"""
cursor.execute(sql_str)

with open("compute/content_popularity.json", "w") as json_file:
    for row in cursor:
        str = '{' + f'"movieId": {row[0]}, "popularity": {row[1]}' + '}\n'
        # print(str, end='')
        str_json = {cursor.description[0].name: row[0], cursor.description[1].name: row[1]}
        print(str_json)
        json_file.write(json.dumps(str_json)+'\n')
    
cursor.close()

In [ ]:
cat "compute/content_popularity.json"